### Pre-talk notes for Speaker!
* get API keys

During talk:
* Run Jupyter from Terminal
* Log in to UKDS Reddit acount
* Open the [Reddit API Dashboard](https://old.reddit.com/prefs/apps/)
* Share public link -> https://github.com/UKDataServiceOpen/working-with-twitter-data/blob/main/TwarcDemo.ipynb
* Share Binder Link -> TODO
* Make Binder -> TODO

Talk time - TODO

In [41]:
# API KEYS - DELETE AFTER TALK
client_id="REPLACEME"
client_secret="REPLACEME"

# Reddit Scraping Demo
In this notebook we detail the recommended way to collect large amounts of Reddit data. 

Previously we recommended Twitter/X as a fantastic source of social media textual data. Sadly, that's no longer the case.

The Reddit API is a close approvimation, though markedly more complicated. Where a tweet stands on it's own contextless, most Reddit comments are related to the post title, or previous comments in a thread in some way and carry a more inherent bias in that regard.

The good news, the Reddit API is free. We can get roughly 100 comments per request × 60 requests = ~6,000 comments/minute. With clever programming we can theoretically grab all comments from a subreddit.

The best package for reddit-scraping is the Python Reddit API Wrapper, or [PRAW](https://praw.readthedocs.io/en/stable/getting_started/installation.html).

# Installation
You will need Python 3 and pip3 availible on your local machine. I recommend doing this by installing [Anaconda](https://docs.anaconda.com/anaconda/install/index.html)

We can check these exist by typing the following:
```
python
```
which should open a REPL and print out our python version.
And:
```
pip3
```
which should log out the manual for pip3. If these don't happen you will need to install these [here](https://www.python.org/downloads/) which may take some time.

```
pip install praw
```
This finally installs PRAW

In [42]:
# !pip3 install praw
# !pip3 install pandas
# !pip3 install openpyxl

In [43]:
# You will have to install PRAW if you haven't already
import praw
# This checks is PRAW is installed
print(praw.__version__)

7.8.1


In [44]:
# Install other helpful packages
import pandas as pd
import openpyxl
from datetime import datetime, timezone

# API Keys
In order to use this to scrape the Reddit API, you will need a Reddit account and API key:
1. Make a Reddit account [here](reddit.com/register).
2. Make a Reddit API key [here](reddit.com/prefs/apps).
3. Click "are you a developer? Make an app..."
4. Mark it as a Web app
5. Fill in all details with a name, description, and URLs relating to your project.

You should see two values:
1. Client ID
2. Client Secret

We won't go into much detail on these, but these two values would give complete access to your reddit account. If you are using a personal account and care about the history of it do not share these EVER. I'd recommend making a new reddit account for research processes.

# Get all comments from a single reddit post

In [45]:
# 1. Authenticate (replace with your credentials)
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="python:testscript:v0.1 (by /u/myusername)",
)

# 2. Get the submission by URL or ID
url = "https://old.reddit.com/r/ghana/comments/18b74n9/i_would_love_to_visit_ghana_what_should_i_know/"
submission = reddit.submission(url=url)

# 3. Load all comments
submission.comments.replace_more(limit=None)

# 4. Flatten the tree into a list
all_comments = submission.comments.list()

# 5. Print them
for comment in all_comments:
    print(comment.body)

Thanks OP for your submission. This sub is heavily moderated by Auto Mod and your post may be mistakenly removed automatically. Please send a message to the mods or u/JuliusCeaserBoneHead for manual approval. Before you do that, make sure your post does not break any of r/Ghana rules especially rule 4 (No Self Promotion).


*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/ghana) if you have any questions or concerns.*
Travelling here between June - September means you'll meet pretty mild weather. Pack enough comfortable tshirts and shorts to accommodate for multiple changes a day. Don't stay in Accra throughout your visit. 

Go to :

-Akosombo (see the Volta lake and multiple riverfront villas)
-Tamale (Bole national park and Paga crocodile pond if possible in the upper east)
-then Kumasi on your way back (Lake Bosomtwe, Kumasi royal museum)

The local cuisine isn't bad. Keep an open mind.

Not too sure 

In [46]:

# 2. Get the submission
url = "https://old.reddit.com/r/ghana/comments/18b74n9/i_would_love_to_visit_ghana_what_should_i_know/"
submission = reddit.submission(url=url)

# 3. Load all comments
submission.comments.replace_more(limit=None)
all_comments = submission.comments.list()

# 4. Prepare list of dicts for DataFrame
data = []
for comment in all_comments:
    data.append({
        "author": str(comment.author),
        "score": comment.score,
        "created_utc": datetime.fromtimestamp(comment.created_utc, timezone.utc).replace(tzinfo=None),
        "body": comment.body.replace("\n", " "),
    })
# 5. print comments
for comment in all_comments:
    print(comment.body)

Thanks OP for your submission. This sub is heavily moderated by Auto Mod and your post may be mistakenly removed automatically. Please send a message to the mods or u/JuliusCeaserBoneHead for manual approval. Before you do that, make sure your post does not break any of r/Ghana rules especially rule 4 (No Self Promotion).


*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/ghana) if you have any questions or concerns.*
Travelling here between June - September means you'll meet pretty mild weather. Pack enough comfortable tshirts and shorts to accommodate for multiple changes a day. Don't stay in Accra throughout your visit. 

Go to :

-Akosombo (see the Volta lake and multiple riverfront villas)
-Tamale (Bole national park and Paga crocodile pond if possible in the upper east)
-then Kumasi on your way back (Lake Bosomtwe, Kumasi royal museum)

The local cuisine isn't bad. Keep an open mind.

Not too sure 

In [47]:
# 5. Create DataFrame
df = pd.DataFrame(data)

df.head()

,author,score,created_utc,body
0,AutoModerator,1,2023-12-05 08:00:46,Thanks OP for your submission. This sub is hea...
1,OG_rafiki,20,2023-12-05 08:33:14,Travelling here between June - September means...
2,steepcurve,12,2023-12-05 08:55:53,If you go to pub/club. Keep count on your drin...
3,Adorable_Rub_8257,6,2023-12-07 22:46:28,Reading all the comments here got me smiling. ...
4,MyRockMyRefuge,3,2023-12-05 08:52:48,It’s great to hear you want to visit Ghana. Th...


In [48]:
# 6. Save to CSV
df.to_csv("data/single_post_reddit_comments.csv", index=False, encoding="utf-8")

print(f"Saved {len(df)} comments to reddit_comments.csv")

Saved 42 comments to reddit_comments.csv


In [49]:
#7. Or save to Excel Spreadsheet
df.to_excel("data/single_post_reddit_comments.xlsx", index=False, engine='openpyxl')

print(f"Saved {len(df)} comments to reddit_comments.xlsx")

Saved 42 comments to reddit_comments.xlsx


# Get all comments, from multiple reddit posts
TODO

# Get all comments, from an entire subreddit
TODO